In [25]:
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import cv2
import copy
from scipy import ndimage as ndi
import tqdm

In [26]:
def adjust_gamma(image, gamma=1.0):
    invGamma = 1.0 / gamma
    table = np.array([((l / 255.0) ** invGamma) * 255 for l in np.arange(0, 256)]).astype("uint8")
    # apply gamma correction using the lookup table
    return cv2.LUT(image.astype(np.uint8), table)

In [32]:
def FiloAnalyzer_full(img, threshold_method, erosion, dilation, filopodia_area_threshold, gamma):

    # PREPROCESSING
    if len(img.shape) > 2:
        body = img[:,:,0]
    else:
        body = img
    body[body < 0.3 * np.mean((body))] = 0
    body = cv2.GaussianBlur(body, (5, 5), 1)
    body = adjust_gamma(body, gamma=gamma)

    # SEGMENTATION
    if threshold_method == "automatic":
        test = copy.copy(body)
        body = cv2.adaptiveThreshold(test, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 655, 5)
        body = cv2.morphologyEx(body, cv2.MORPH_ERODE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)))
        body = ndi.binary_fill_holes(body).astype(np.uint8)
        x, _ = cv2.findContours(body, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        body = np.zeros_like(body)
        for i in x:
            mom = cv2.moments(i)
            area = mom['m00']
            if area > 10000 and area < 2000000:
                cv2.drawContours(body, [i], -1, (255, 144, 255), -1)
    elif threshold_method == "triangle":
        test = copy.copy(body)
        hist = np.histogram(body.ravel(), bins=256, range=(0.0, 255))
        ret, body = cv2.threshold(body, 42, 255, cv2.THRESH_BINARY + cv2.THRESH_TRIANGLE)
        s = hist[0][int(ret):int(ret) + 12]
        s[np.where(s == 0)] = np.max(s)
        t = np.argmin(s) + int(ret)
        ret, body = cv2.threshold(test, t, 255, cv2.THRESH_BINARY)
        body = ndi.binary_fill_holes(body).astype(np.uint8)
    else:
        ret, body = cv2.threshold(body, threshold_method, 255, cv2.THRESH_BINARY)
    
    thresholded_image = copy.deepcopy(body)
    
    nucleus = cv2.morphologyEx(body, cv2.MORPH_ERODE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (erosion, erosion)))
    nucleus = cv2.morphologyEx(nucleus, cv2.MORPH_DILATE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (dilation, dilation)))
    body[nucleus != 0] = 0

    opened_image = copy.deepcopy(body)

    # EXTRACT ONLY BIGGEST AND ELONGATED CONTOURS
    result = np.zeros_like(body)
    x, _ = cv2.findContours(body.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for i in x:
        moments = cv2.moments(i)
        area = moments['m00']
        if area > filopodia_area_threshold and area < 200000:
            if moments['m00'] != 0.0:
                ellip = cv2.fitEllipse(i)
                (_, axes, _) = ellip
                major_axis = max(axes)
                minor_axis = min(axes)
                if np.sqrt(1 - (minor_axis * minor_axis) / (major_axis * major_axis)) > 0.7:
                    cv2.drawContours(result, [i], -1, 1, -1)

    return thresholded_image, opened_image, result

In [33]:
import os

def analyze_and_save_images(folder_path):
    for filename in tqdm.tqdm(os.listdir(folder_path)):
        if os.path.isfile(os.path.join(folder_path, filename)) and filename.lower().endswith((".png", ".jpg", ".jpeg", ".tif", ".tiff")):
            # Get the image name without extension
            image_name, _ = os.path.splitext(filename)
            image_to_read = os.path.join(folder_path, filename)
            img = cv2.imread(image_to_read)
            
            # Apply FiloAnalyzer_full function on the image
            result = FiloAnalyzer_full(img, "triangle", 20, 20, 20, 1)[2] * 255
            
            # Create the output filename with .tif extension
            output_filename = os.path.join(r'C:\Users\ricca\Desktop\Thesis\toolsData\FiloAnalyzer320', f"{image_name}.tif")
            
            cv2.imwrite(output_filename, result)

analyze_and_save_images('dataset\images')


100%|██████████| 245/245 [00:29<00:00,  8.32it/s]
